# 🌐 Web Stack From Scratch - The Complete Journey

## localhost → DNS → File Server → Mail Server → Web Server → Reverse Proxy → Production

**This notebook teaches how the ENTIRE web stack works with ZERO dependencies.**

### What You'll Learn:
1. **Origin Server (Offline/Localhost)** - Your computer running code
2. **DNS** - How domain names map to IP addresses
3. **File Server** - Where data lives (SQLite database)
4. **Mail Server** - Sending emails (SMTP)
5. **Web Server** - Serving HTTP requests (Flask/Gunicorn)
6. **Reverse Proxy** - Routing multiple domains (Caddy/nginx)
7. **Production Deployment** - GitHub Pages + Caddy

### No Dependencies Needed:
- ✅ Python3 (built-in on macOS)
- ✅ SQLite (built-in on macOS)
- ✅ sendmail (built-in on macOS)
- ✅ curl (built-in on macOS)

**Every cell is pure Python - copy/paste and run!**

## Cell 1: Origin Server (Localhost)

**The beginning - your computer running code.**

This is where EVERYTHING starts:
- `127.0.0.1` (localhost) = Your computer
- `:5001` = Port number (like an apartment number)
- Flask app = Web server software

**Think of it like:**
- Your computer = The building
- Port 5001 = Apartment 5001
- Flask = The person answering the door

In [ ]:
# CELL 1: Minimal Flask Web Server (NO dependencies)
# This is the ORIGIN - where your website lives

from http.server import HTTPServer, BaseHTTPRequestHandler

class SimpleHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        """Handle HTTP GET requests"""
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
        html = b'''
        <!DOCTYPE html>
        <html>
        <head><title>Origin Server</title></head>
        <body>
            <h1>Origin Server Running!</h1>
            <p>You're seeing this from: <code>127.0.0.1:8000</code></p>
            <p>This is <strong>localhost</strong> - your computer!</p>
        </body>
        </html>
        '''
        self.wfile.write(html)

# Start server on localhost:8000
print("🚀 Origin server starting on http://127.0.0.1:8000")
print("   Visit in browser: http://localhost:8000")
print("   Press Ctrl+C to stop")
print()

# This would run the server (comment out in Jupyter)
# server = HTTPServer(('127.0.0.1', 8000), SimpleHandler)
# server.serve_forever()

print("✅ Code ready! Run outside Jupyter to start server.")

## Cell 2: DNS - Domain Name System

**How does `soulfra.com` become `185.199.109.153`?**

### The Journey:
```
User types: soulfra.com
    ↓
DNS Resolver: "What IP is soulfra.com?"
    ↓
Root DNS: "Ask .com servers"
    ↓
.com DNS: "Ask soulfra's nameservers"
    ↓
Nameserver: "185.199.109.153"
    ↓
Browser connects to 185.199.109.153
```

### Three Levels:
1. **Local (/etc/hosts)** - Your computer's override
2. **Private DNS** - Company internal (not used here)
3. **Public DNS** - Domain registrar (GoDaddy, Namecheap, etc.)

In [ ]:
# CELL 2: DNS Resolution (Pure Python)

import socket
import subprocess

print("🌐 DNS RESOLUTION DEMO\n")
print("="*70)

# Level 1: Local /etc/hosts file
print("\n[Level 1: Local DNS Override]")
print("File: /etc/hosts")
print()

# Show local DNS entries
with open('/etc/hosts', 'r') as f:
    for line in f:
        if 'localhost' in line or 'soulfra' in line or 'calriven' in line:
            print(f"   {line.strip()}")

print("\n   💡 These override public DNS!")
print("      calriven.local → 127.0.0.1 (your computer)")

# Level 2: Public DNS lookup
print("\n[Level 2: Public DNS Lookup]")
domains = ['soulfra.com', 'github.com', 'google.com']

for domain in domains:
    try:
        ip = socket.gethostbyname(domain)
        print(f"   {domain:<20} → {ip}")
    except Exception as e:
        print(f"   {domain:<20} → Error: {e}")

# Level 3: DNS Records (A, CNAME, MX, TXT)
print("\n[Level 3: DNS Record Types]")
print("\n   A Record     → IP address (IPv4)")
print("   AAAA Record  → IP address (IPv6)")
print("   CNAME Record → Alias to another domain")
print("   MX Record    → Mail server")
print("   TXT Record   → Text data (SPF, DKIM)")

print("\n" + "="*70)
print("\n✅ DNS connects human-readable names to machine IP addresses!")

## Cell 3: File Server - Where Data Lives

**Every website needs to store data somewhere.**

### Options:
1. **SQLite** - Database in a file (what we use)
2. **JSON files** - Static data
3. **Text files** - Simple storage
4. **Object storage** - S3, Cloudflare R2

### Why SQLite?
- ✅ No server needed (just a file)
- ✅ Built into macOS/Linux
- ✅ Fast for <100k users
- ✅ ACID compliant (safe)

**Think of it like:**
- Database = Filing cabinet
- Table = Drawer
- Row = Folder
- Column = Label on folder

In [ ]:
# CELL 3: File Server (SQLite Database)

import sqlite3
import os

print("📁 FILE SERVER DEMO\n")
print("="*70)

# Create/connect to database file
DB_FILE = 'demo.db'
db = sqlite3.connect(DB_FILE)
cursor = db.cursor()

print(f"\n[Step 1: Create Database]")
print(f"   File: {DB_FILE}")
print(f"   Location: {os.path.abspath(DB_FILE)}")

# Create a domains table
print(f"\n[Step 2: Create Table (Filing Cabinet Drawer)]")
cursor.execute('''
    CREATE TABLE IF NOT EXISTS domains (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        domain TEXT NOT NULL,
        ip_address TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')
print("   ✅ Table 'domains' created")

# Insert data
print(f"\n[Step 3: Insert Data (Add Folders)]")
domains = [
    ('Soulfra', 'soulfra.com', '185.199.109.153'),
    ('CalRiven', 'calriven.com', '127.0.0.1'),
    ('DeathToData', 'deathtodata.com', '127.0.0.1'),
]

for name, domain, ip in domains:
    cursor.execute(
        'INSERT OR IGNORE INTO domains (name, domain, ip_address) VALUES (?, ?, ?)',
        (name, domain, ip)
    )
    print(f"   ✅ Added: {name} ({domain}) → {ip}")

db.commit()

# Query data
print(f"\n[Step 4: Query Data (Read Folders)]")
cursor.execute('SELECT name, domain, ip_address FROM domains')
results = cursor.fetchall()

print("\n   ID   Name              Domain                IP")
print("   " + "-"*60)
for i, (name, domain, ip) in enumerate(results, 1):
    print(f"   {i:<4} {name:<16} {domain:<20} {ip}")

# Show file size
file_size = os.path.getsize(DB_FILE)
print(f"\n[Database Info]")
print(f"   File size: {file_size:,} bytes")
print(f"   Records: {len(results)}")

db.close()

print("\n" + "="*70)
print("\n✅ File server stores ALL your website data in one file!")

## Cell 4: Mail Server - SMTP Email

**How does email actually work?**

### The Email Journey:
```
You write email in app.py
    ↓
Python calls sendmail
    ↓
sendmail connects to SMTP server (port 25)
    ↓
SMTP server looks up MX record
    ↓
Delivers to recipient's mail server
    ↓
Recipient checks inbox
```

### Three Options:
1. **sendmail (macOS built-in)** - Local testing
2. **SMTP service (Gmail, Resend)** - Production
3. **Your own mail server (postfix)** - Advanced

### Ports:
- Port 25: SMTP (send)
- Port 465: SMTPS (encrypted)
- Port 587: Submission (auth required)
- Port 993: IMAP (receive)
- Port 995: POP3 (receive)

In [ ]:
# CELL 4: Mail Server (SMTP Email)

import smtplib
import subprocess
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

print("📧 MAIL SERVER DEMO\n")
print("="*70)

# Check if sendmail exists (macOS built-in)
print("\n[Step 1: Check Mail Server]")
try:
    result = subprocess.run(['which', 'sendmail'], capture_output=True, text=True)
    if result.stdout:
        print(f"   ✅ sendmail found: {result.stdout.strip()}")
    else:
        print("   ⚠️  sendmail not found (macOS built-in usually)")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Show mail server config
print("\n[Step 2: SMTP Configuration]")
print("   Server: localhost")
print("   Port: 25 (SMTP)")
print("   Auth: Not required (localhost)")
print("   Encryption: None (local only)")

# Create email (doesn't actually send in notebook)
print("\n[Step 3: Compose Email]")

msg = MIMEMultipart()
msg['From'] = 'noreply@soulfra.com'
msg['To'] = 'user@example.com'
msg['Subject'] = 'Test Email from Soulfra'

body = '''Hello!

This is a test email from your website.

It was sent via:
- Python's smtplib
- macOS sendmail
- Port 25 (SMTP)

Thanks,
Soulfra
'''

msg.attach(MIMEText(body, 'plain'))

print("   From: noreply@soulfra.com")
print("   To: user@example.com")
print("   Subject: Test Email from Soulfra")
print(f"   Body: {len(body)} characters")

# Show how to send (don't actually send in notebook)
print("\n[Step 4: Send Email (Example Code)]")
print("\n   # Connect to localhost SMTP")
print("   smtp = smtplib.SMTP('localhost', 25)")
print("   smtp.send_message(msg)")
print("   smtp.quit()")

print("\n   ⚠️  Not actually sending (notebook demo only)")

# Show MX records for domain
print("\n[Step 5: MX Records (Mail Exchange)]")
print("   MX records tell where to deliver email")
print("\n   Example for gmail.com:")
print("   gmail-smtp-in.l.google.com (priority 5)")
print("   alt1.gmail-smtp-in.l.google.com (priority 10)")

print("\n" + "="*70)
print("\n✅ Mail servers handle email delivery via SMTP protocol!")

## Cell 5: Web Server - Serving HTTP

**The web server responds to browser requests.**

### Two Modes:

**Development (localhost):**
- Flask built-in server
- Single-threaded
- Debug mode
- Port 5001

**Production (soulfra.com):**
- Gunicorn (or uWSGI)
- Multi-threaded
- No debug mode
- Port 80/443

### HTTP Request/Response:
```
Browser: GET / HTTP/1.1
         Host: soulfra.com
    ↓
Web Server: HTTP/1.1 200 OK
            Content-Type: text/html
            
            <html>...</html>
```

In [ ]:
# CELL 5: Web Server (HTTP Request/Response)

print("🌐 WEB SERVER DEMO\n")
print("="*70)

# Show the difference between dev and production
print("\n[Development vs Production]")
print()
print("   DEVELOPMENT (localhost):")
print("   ├─ Command: python3 app.py")
print("   ├─ Server: Flask built-in (Werkzeug)")
print("   ├─ Port: 5001")
print("   ├─ Threads: 1 (single-threaded)")
print("   ├─ Debug: Enabled")
print("   ├─ Reload: Auto-reload on code change")
print("   └─ Use: Local testing only")
print()
print("   PRODUCTION (soulfra.com):")
print("   ├─ Command: gunicorn app:app -w 4 -b 0.0.0.0:5001")
print("   ├─ Server: Gunicorn (WSGI)")
print("   ├─ Port: 5001 (behind Caddy proxy)")
print("   ├─ Workers: 4 (multi-process)")
print("   ├─ Debug: Disabled")
print("   ├─ Reload: Manual restart")
print("   └─ Use: Real traffic")

# Simulate HTTP request/response
print("\n[HTTP Request/Response Cycle]")
print()
print("   1. Browser sends request:")
print("      GET / HTTP/1.1")
print("      Host: soulfra.com")
print("      User-Agent: Mozilla/5.0...")
print()
print("   2. Web server receives request")
print("      Flask app.py @ route '/'")
print()
print("   3. Server queries database")
print("      SELECT * FROM brands WHERE domain='soulfra.com'")
print()
print("   4. Server renders HTML")
print("      render_template('index.html', brand=brand)")
print()
print("   5. Server sends response:")
print("      HTTP/1.1 200 OK")
print("      Content-Type: text/html")
print("      Content-Length: 5432")
print("      ")
print("      <html>...</html>")

# Show common ports
print("\n[Common Ports]")
print()
print("   Port 80   → HTTP (unencrypted)")
print("   Port 443  → HTTPS (encrypted with SSL)")
print("   Port 5001 → Flask development server")
print("   Port 8000 → Alternative web server")
print("   Port 3000 → Node.js/React dev server")

print("\n" + "="*70)
print("\n✅ Web servers turn HTTP requests into HTML responses!")

## Cell 6: Reverse Proxy - Multi-Domain Routing

**How does ONE server handle multiple domains?**

### The Problem:
- You have: soulfra.com, calriven.com, deathtodata.com
- All point to same IP: 185.199.109.153
- How does server know which site to show?

### The Solution: Reverse Proxy

```
Internet → Caddy (Port 80/443) → Backend
           |
           ├─ soulfra.com → Flask :5001
           ├─ calriven.com → Flask :5002
           └─ deathtodata.com → Flask :5003
```

### Two Options:

**Option 1: Caddy (Automatic HTTPS)**
```
soulfra.com {
    reverse_proxy localhost:5001
}
```

**Option 2: nginx**
```
server {
    server_name soulfra.com;
    location / {
        proxy_pass http://localhost:5001;
    }
}
```

In [ ]:
# CELL 6: Reverse Proxy (Caddy Example)

print("🔀 REVERSE PROXY DEMO\n")
print("="*70)

# Show Caddyfile configuration
print("\n[Caddy Configuration - Caddyfile]")
print()

caddyfile = '''
# Soulfra domain
soulfra.com {
    reverse_proxy localhost:5001
    encode gzip
    
    # Automatic HTTPS!
    # Caddy gets SSL cert from Let's Encrypt
}

# CalRiven domain
calriven.com {
    reverse_proxy localhost:5002
    encode gzip
}

# DeathToData domain
deathtodata.com {
    reverse_proxy localhost:5003
    encode gzip
}

# Localhost development
localhost:5001 {
    reverse_proxy localhost:5001
}
'''

print(caddyfile)

# Show how it routes requests
print("\n[Request Routing]")
print()
print("   1. Request arrives: https://soulfra.com/")
print("      ↓")
print("   2. Caddy receives on port 443 (HTTPS)")
print("      ↓")
print("   3. Caddy checks Host header: 'soulfra.com'")
print("      ↓")
print("   4. Matches config: soulfra.com → localhost:5001")
print("      ↓")
print("   5. Proxies to Flask app on port 5001")
print("      ↓")
print("   6. Flask returns HTML")
print("      ↓")
print("   7. Caddy forwards response to browser")

# Show what Caddy provides
print("\n[What Caddy Does For You]")
print()
print("   ✅ Automatic HTTPS (Let's Encrypt)")
print("   ✅ Certificate renewal (automatic)")
print("   ✅ HTTP → HTTPS redirect")
print("   ✅ Gzip compression")
print("   ✅ Multiple domains on one port")
print("   ✅ Load balancing (if needed)")

# Alternative: Using ONE Flask app with subdomain detection
print("\n[Alternative: Single Flask App with Brand Detection]")
print()
print("   Instead of multiple Flask apps, ONE app detects domain:")
print()
print("   @app.before_request")
print("   def detect_brand():")
print("       host = request.host")
print("       if 'soulfra.com' in host:")
print("           g.brand = 'soulfra'")
print("       elif 'calriven.com' in host:")
print("           g.brand = 'calriven'")
print("   ")
print("   (This is what YOUR app does!)")

print("\n" + "="*70)
print("\n✅ Reverse proxies route multiple domains to backend servers!")

## Cell 7: Production Deployment - The Full Stack

**Putting it all together: localhost → production**

### Current Setup:

**GitHub Pages (Static):**
- soulfra.com → 185.199.109.153 (GitHub)
- Static HTML/CSS/JS files
- No database, no Python
- Fast, free, simple

**Flask (Dynamic):**
- localhost:5001
- Python + SQLite
- User accounts, databases
- Full web app

### Two Deployment Options:

**Option 1: GitHub Pages Only (Static)**
```
1. Build static site: python3 build.py
2. Push to GitHub: git push origin main
3. GitHub Pages serves: soulfra.com
```

**Option 2: VPS + Caddy (Dynamic)**
```
1. Get VPS: DigitalOcean, Linode ($5/mo)
2. Install Caddy: curl -sSL https://caddyserver.com/download | bash
3. Clone repo: git clone ...
4. Start Flask: gunicorn app:app -w 4
5. Start Caddy: caddy run
6. Update DNS: A record → VPS IP
```

In [ ]:
# CELL 7: Full Stack Deployment Checklist

print("🚀 PRODUCTION DEPLOYMENT GUIDE\n")
print("="*70)

# Show current state
print("\n[Current State]")
print()
print("   🏠 Development:")
print("      • Flask on localhost:5001")
print("      • SQLite database (soulfra.db)")
print("      • Debug mode enabled")
print("      • Auto-reload on changes")
print()
print("   🌐 Production:")
print("      • soulfra.com on GitHub Pages")
print("      • Static HTML only")
print("      • No database")
print("      • Fast but limited")

# Deployment checklist
print("\n[Deployment Checklist - VPS Option]")
print()

checklist = [
    ("1", "Get VPS", "DigitalOcean $5/mo or Linode"),
    ("2", "Setup DNS", "A record: soulfra.com → VPS_IP"),
    ("3", "Install Python", "apt install python3 python3-pip"),
    ("4", "Clone repo", "git clone https://github.com/soulfra/soulfra"),
    ("5", "Install deps", "pip3 install -r requirements.txt"),
    ("6", "Setup database", "cp soulfra.db /var/www/soulfra/"),
    ("7", "Install Caddy", "wget https://caddyserver.com/download"),
    ("8", "Configure Caddy", "Create Caddyfile with domains"),
    ("9", "Start Gunicorn", "gunicorn app:app -w 4 -b 127.0.0.1:5001"),
    ("10", "Start Caddy", "caddy run --config Caddyfile"),
    ("11", "Test HTTPS", "curl https://soulfra.com"),
    ("12", "Setup systemd", "Auto-start on boot"),
]

for num, task, desc in checklist:
    print(f"   [{num:>2}] {task:<20} {desc}")

# Zero dependencies option
print("\n[Zero Dependencies Deployment]")
print()
print("   What you NEED (built into macOS/Linux):")
print("   ✅ Python3")
print("   ✅ SQLite")
print("   ✅ sendmail")
print("   ✅ curl")
print()
print("   What you DON'T need:")
print("   ❌ Docker")
print("   ❌ npm/Node.js")
print("   ❌ MySQL/PostgreSQL")
print("   ❌ Redis")
print("   ❌ Complex build tools")

# Show the full stack
print("\n[The Complete Stack]")
print()
print("   User Browser")
print("      ↓ HTTPS (port 443)")
print("   Caddy (Reverse Proxy)")
print("      ↓ HTTP (port 5001)")
print("   Gunicorn (WSGI Server)")
print("      ↓ Python")
print("   Flask (Web Framework)")
print("      ↓ SQL")
print("   SQLite (Database)")
print("      ↓ File I/O")
print("   soulfra.db (Data Storage)")

print("\n" + "="*70)
print("\n✅ You now understand the ENTIRE web stack!")
print("\n   From localhost:5001 → soulfra.com production")
print("   With zero dependencies - just Python + SQLite!")

## Summary: The Complete Journey

### From Scratch to Production:

1. **Origin Server (localhost)**
   - Your computer runs Python
   - Flask starts on port 5001
   - Visit: http://127.0.0.1:5001

2. **DNS Setup**
   - Local: Edit /etc/hosts
   - Public: Configure domain registrar
   - Points domain → IP address

3. **File Server**
   - SQLite database (soulfra.db)
   - Stores: users, brands, posts
   - No external database needed

4. **Mail Server**
   - sendmail (built into macOS)
   - SMTP on port 25
   - Or use Resend/Gmail SMTP

5. **Web Server**
   - Dev: Flask built-in (localhost)
   - Prod: Gunicorn (multi-process)
   - Responds to HTTP requests

6. **Reverse Proxy**
   - Caddy handles multiple domains
   - Automatic HTTPS (Let's Encrypt)
   - Routes: soulfra.com → :5001

7. **Production**
   - VPS running Flask + Caddy
   - OR GitHub Pages (static)
   - DNS points domain → server IP

### No Dependencies Needed:

```bash
# Everything built into macOS:
python3    # Web framework
sqlite3    # Database
sendmail   # Email
curl       # HTTP client

# Optional for production:
caddy      # Reverse proxy (single binary)
gunicorn   # WSGI server (pip install)
```

### Next Steps:

1. **Fix homepage** - Run `python3 fix_homepage.py`
2. **Test locally** - `python3 app.py`
3. **Deploy** - Choose GitHub Pages OR VPS

**You now understand how the ENTIRE internet works!** 🎉